In [1]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset

import os
import time

from DL_Models.torch_models.SalientSleepNet.SalientSleepNet import U2
from config import config, create_folder
from hyperparameters import all_models
from DL_Models.torch_models.torch_utils.DiceLoss import DiceLoss

from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay, classification_report, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import scipy

In [2]:
def create_dataloader(X, y, batch_size, drop_last=True, num_workers=10):
    """
    Input: X, y of type np.array
    Return: pytorch dataloader containing the dataset of X and y that returns batches of size
    batch_size
    """
    y = y.astype("float32")
    # Transform np.array to torch flaot tensor
    tensor_x = torch.as_tensor(X).float()
    tensor_y = torch.as_tensor(y).float()

    if len(tensor_y.size()) > 2:
        tensor_y = tensor_y.squeeze()

    # Unsqueeze channel direction for eegNet model
    # if model_name == 'EEGNet':
    #     logging.info(f"Unsqueeze data for eegnet")
    #     tensor_x = tensor_x.unsqueeze(1)
    # Log the shapes
    print(f"Tensor x size: {tensor_x.size()}")
    print(f"Tensor y size: {tensor_y.size()}")
    # Set device
    # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # Create dataset and dataloader
    dataset = TensorDataset(tensor_x, tensor_y)
    return DataLoader(dataset, batch_size=batch_size, drop_last=drop_last, num_workers=num_workers)

In [3]:
data = np.load("../test_npz/20240209_M4_MAT.npz")

test_dataloader=create_dataloader(data['EEG'], data['labels'], 32)

Tensor x size: torch.Size([518, 2000, 1])
Tensor y size: torch.Size([518, 2000])


In [4]:
config['retrain']=False
config['load_experiment_dir']='weighted_dice_run3/'

In [5]:
config

{'task': 'Segmentation_task',
 'dataset': 'mice_eeg',
 'preprocessing': 'min',
 'include_DL_models': True,
 'include_your_models': False,
 'biased_sampling': False,
 'window_len': 2000,
 'data_dir': '/standard/ivy-hip-rderi/ss4yd/DETRtime/Baselines/',
 'dice_weights': array([ 1.05263158, 20.        ]),
 'log_dir': './runs/',
 'root_dir': '.',
 'retrain': False,
 'pretrained': False,
 'save_models': True,
 'load_experiment_dir': 'weighted_dice_run3/',
 'all_EEG_file': 'Segmentation_task_with_mice_eeg_synchronised_min',
 'framework': 'pytorch',
 'learning_rate': 0.001,
 'early_stopping': True,
 'patience': 10,
 'trainX_variable': 'EEG',
 'trainY_variable': 'labels'}

In [6]:
create_folder()

In [7]:
config

{'task': 'Segmentation_task',
 'dataset': 'mice_eeg',
 'preprocessing': 'min',
 'include_DL_models': True,
 'include_your_models': False,
 'biased_sampling': False,
 'window_len': 2000,
 'data_dir': '/standard/ivy-hip-rderi/ss4yd/DETRtime/Baselines/',
 'dice_weights': array([ 1.05263158, 20.        ]),
 'log_dir': './runs/',
 'root_dir': '.',
 'retrain': False,
 'pretrained': False,
 'save_models': True,
 'load_experiment_dir': 'weighted_dice_run3/',
 'all_EEG_file': 'Segmentation_task_with_mice_eeg_synchronised_min',
 'framework': 'pytorch',
 'learning_rate': 0.001,
 'early_stopping': True,
 'patience': 10,
 'trainX_variable': 'EEG',
 'trainY_variable': 'labels',
 'model_dir': './runs/weighted_dice_run3/',
 'checkpoint_dir': './runs/weighted_dice_run3/checkpoint/',
 'info_log': './runs/weighted_dice_run3//inference_info_1765202506.log',
 'batches_log': './runs/weighted_dice_run3//inference_batches_1765202506.log'}

In [8]:
models = all_models[config["task"]][config["dataset"]][config["preprocessing"]]
models

{'PyramidalCNN': [DL_Models.Ensemble.Ensemble,
  {'model_name': 'PyramidalCNN',
   'nb_models': 1,
   'loss': 'dice-loss',
   'batch_size': 32,
   'input_shape': (2000, 1),
   'output_shape': 2000,
   'nb_outlayer_channels': 2,
   'kernel_size': 32,
   'epochs': 10,
   'nb_filters': 16,
   'verbose': True,
   'use_residual': False,
   'depth': 5}],
 'Xception': [DL_Models.Ensemble.Ensemble,
  {'model_name': 'Xception',
   'nb_models': 1,
   'loss': 'dice-loss',
   'batch_size': 32,
   'input_shape': (2000, 1),
   'output_shape': 2000,
   'nb_outlayer_channels': 2,
   'kernel_size': 32,
   'epochs': 10,
   'nb_filters': 128,
   'verbose': True,
   'use_residual': True,
   'depth': 12}],
 'InceptionTime': [DL_Models.Ensemble.Ensemble,
  {'model_name': 'InceptionTime',
   'nb_models': 1,
   'loss': 'dice-loss',
   'batch_size': 32,
   'input_shape': (2000, 1),
   'output_shape': 2000,
   'nb_outlayer_channels': 2,
   'kernel_size': 32,
   'epochs': 10,
   'nb_filters': 32,
   'verbose': T

In [9]:
# N=1
# for name, model in models.items():

#     for i in range(N):
#         # create the model with the corresponding parameters
#         path = config["checkpoint_dir"] + "run" + str(i + 1) + "/"
#         trainer = model[0](path, **model[1])
#         break
#     break

# trainer

# trainer.load(path)

# model

models.keys()

dict_keys(['PyramidalCNN', 'Xception', 'InceptionTime', 'CNN', 'EEGNet', 'SalientSleepNet', 'UNet', 'ConvLSTM', 'LSTM', 'biLSTM'])

In [10]:
save_dir='./weighted_dice_results_csvs_3'
os.makedirs(save_dir, exist_ok=True)
for model_name in models.keys():
    print(model_name)
    path=config['checkpoint_dir']+"run1/"
    trainer=models[model_name][0](path, **models[model_name][1])
    trainer.load(path)

    dice_weights = config["dice_weights"]
    trainer.loss_fn = DiceLoss(weight=torch.tensor(dice_weights), p=2, reduction="mean")

    # trainer=trainer.to('cuda')
    for i, fn in enumerate(os.listdir('../test_npz/')):
        print(fn)
        data = np.load(f"../test_npz/{fn}")

        test_dataloader=create_dataloader(data['EEG'], data['labels'], 32, num_workers=1)
        correct_pred, predictions, all_probs=trainer.predict(test_dataloader)

    #     mdic={'predicted_label':predictions, 'ecog':data['EEG'], 'predicted_probs':all_probs,'actual_label':correct_pred}
    #     with open(f'predicted_{i.split('.')[0]}.mat', 'wb') as file:
    #         scipy.io.savemat(file, mdic, appendmat=True)
    #     print('\n')

        classifier_score_df = pd.DataFrame(
                                classification_report(correct_pred,
                                                     predictions,
                                                     target_names=['Non-Seizure',
                                                                   'Seizure'],
                                                     digits=2,
                                                     output_dict=True))

        classifier_score_df.index.name = 'Metric'
        classifier_score_df['Classifier'] = model_name
        classifier_score_df['MID'] = fn
        classifier_score_df = classifier_score_df.set_index('Classifier', append=True)

        if i == 0:
            all_scores = classifier_score_df

        else:
            all_scores = pd.concat([all_scores, classifier_score_df])

    all_scores.to_csv(f'{save_dir}/{model_name}_results.csv')



PyramidalCNN
20240209_M3_MAT.npz
Tensor x size: torch.Size([491, 2000, 1])
Tensor y size: torch.Size([491, 2000])


  0%|          | 0/15 [00:00<?, ?it/s]

Test performance: 
{'0': {'precision': 0.944994633571957, 'recall': 1.0, 'f1-score': 0.9717195279212538, 'support': 906883.0}, '1': {'precision': 1.0, 'recall': 0.0062127002654517385, 'f1-score': 0.01234868187176081, 'support': 53117.0}, 'accuracy': 0.9450135416666666, 'macro avg': {'precision': 0.9724973167859785, 'recall': 0.5031063501327259, 'f1-score': 0.49203410489650734, 'support': 960000.0}, 'weighted avg': {'precision': 0.948038091955872, 'recall': 0.9450135416666666, 'f1-score': 0.918637339140409, 'support': 960000.0}}
20240209_M4_MAT.npz
Tensor x size: torch.Size([518, 2000, 1])
Tensor y size: torch.Size([518, 2000])


  0%|          | 0/16 [00:00<?, ?it/s]

Test performance: 
{'0': {'precision': 0.9578525098079652, 'recall': 1.0, 'f1-score': 0.978472591790804, 'support': 975386.0}, '1': {'precision': 1.0, 'recall': 0.11714732381618463, 'f1-score': 0.20972582813161722, 'support': 48614.0}, 'accuracy': 0.9580869140625, 'macro avg': {'precision': 0.9789262549039826, 'recall': 0.5585736619080923, 'f1-score': 0.5940992099612106, 'support': 1024000.0}, 'weighted avg': {'precision': 0.9598534454409687, 'recall': 0.9580869140625, 'f1-score': 0.9419766394777886, 'support': 1024000.0}}
20240209_M5_MAT.npz
Tensor x size: torch.Size([515, 2000, 1])
Tensor y size: torch.Size([515, 2000])


...

/standard/ivy-hip-rderi/ss4yd/.conda/envs/ood_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/standard/ivy-hip-rderi/ss4yd/.conda/envs/ood_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/standard/ivy-hip-rderi/ss4yd/.conda/envs/ood_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavio

  0%|          | 0/19 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f0f600b6520>
Traceback (most recent call last):
  File "/standard/ivy-hip-rderi/ss4yd/.conda/envs/ood_env/lib/python3.13/site-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/standard/ivy-hip-rderi/ss4yd/.conda/envs/ood_env/lib/python3.13/site-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
    if w.is_alive():
  File "/standard/ivy-hip-rderi/ss4yd/.conda/envs/ood_env/lib/python3.13/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f0f600b6520>
Traceback (most recent call last):
  File "/standard/ivy-hip-rderi/ss4yd/.conda/envs/ood_env/lib/python3.13/site-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown

Test performance: 
{'0': {'precision': 0.9649514802631579, 'recall': 1.0, 'f1-score': 0.9821631627605644, 'support': 1173381.0}, '1': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 42619.0}, 'accuracy': 0.9649514802631579, 'macro avg': {'precision': 0.48247574013157896, 'recall': 0.5, 'f1-score': 0.4910815813802822, 'support': 1216000.0}, 'weighted avg': {'precision': 0.9311313592620595, 'recall': 0.9649514802631579, 'f1-score': 0.9477397977657516, 'support': 1216000.0}}


/standard/ivy-hip-rderi/ss4yd/.conda/envs/ood_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/standard/ivy-hip-rderi/ss4yd/.conda/envs/ood_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/standard/ivy-hip-rderi/ss4yd/.conda/envs/ood_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavio

In [11]:
path

'./runs/weighted_dice_run3/checkpoint/run1/'

In [12]:
'./runs/'

'./runs/'